In [7]:
import numpy as np
import matplotlib.pyplot as plt
from math import exp, sqrt

Black and Scholes assumed:
1. A continuously-compounded interest rate of r.
2. Geometric Brownian motion dynamics for the stock price, St
, so that, 
$ S_t = S_0e^{(µ−σ^2/2)t+σW_t} $
where $W_t$
is a standard Brownian motion.
3. The stock pays a dividend yield of c.
4. Continuous trading with no transactions costs and short-selling allowed

The Black-Scholes formula for the price of a European call option with strike
K and maturity T is given by
$C_0 = S_0e^{−cT} * N(d1) − Ke^{−rT} * N(d2) $
where
$d1 = (log(S_0/K) + (r − c + σ^2/2)T)/ σ√T $ ,
and
d2 = d1 − σ√T
and N(d) = P(N(0, 1) ≤ d) - normal distribution 



Often specify a binomial model in terms of Black-Scholes parameters:
1. r, the continuously compounded interest rate.
2. σ, the annualized volatility.

In [8]:
def Parameter(T,n,sigma,r,c):
    """Parameter calculation"""  
    '''T is the time to maturity and n is the number of periods'''
    '''sigma is the volatility'''
    '''r is the the continuously compounded interest rate'''
    dt = T/n
    u = exp(sigma * sqrt(dt))
    d = 1/u
    
    q1 = (exp((r-c)*dt)-d)/(u-d)
    q2 = 1-q1
    R = exp(r*dt)
    
    return (u, d, R, q1, q2)

Once the above parameters are calculated, the multiperiod bionomial model is calibrated by using them and the latter can be used to price options

In [9]:
def bionomial_model(T,n,sigma,r,c, S):
    '''S is the stock price at t = 0'''
    '''u and d are the multiplier with which the stock price "S" moves up and down respectively'''
    '''R is risk free interest rate'''
    '''n is the number of time periods until maturity'''
    
    u, d, R, q1, q2 = Parameter(T,n,sigma,r,c)
    #Calculate stock trr, ie. stock prices at every time interval
    stockTree = np.zeros((n+1, n+1))  
    stockTree[0,0] = S
    for i in range(1,n+1):
        stockTree[0,i] = stockTree[0, i-1]*u
        for j in range(1,n+1):
            stockTree[j,i] = stockTree[j-1, i-1]*d
    return stockTree
    

In [10]:
def european_call(T,n,sigma,r,c, S, K):
    '''u, d, R, S are described as above'''
    '''K is the strike price'''
    u, d, R, q1, q2 = Parameter(T,n,sigma,r,c)
    stockTree = bionomial_model(T,n,sigma,r,c)
    
    # compute the option tree
    optionTree = np.zeros((n+1,n+1))
    for j in range(n+1):
        optionTree[j, n] = max(0,  (stockTree[j, n]-K))
        
    q1 = calculate_risk_neutral_prob(u, d, R) 
    q2 = 1 - q1
    '''q1 and q2 are called risk neutral probabilities'''
    for i in range(n-1,-1,-1):
        for j in range(i+1):
            optionTree[j, i] = (q1 * optionTree[j, i+1] + q2 * optionTree[j+1, i+1])/R
   
                
    return optionTree

In [11]:
def european_put(T,n,sigma,r,c, S, K):
    '''u, d, R, S are described as above'''
    '''K is the strike price'''
    u, d, R, q1, q2 = Parameter(T,N,sigma,r,c)
    stockTree = bionomial_model(T,n,sigma,r,c)
    
    # compute the option tree
    optionTree = np.zeros((n+1,n+1))
    for j in range(n+1):
        optionTree[j, n] = max(0,  (K - stockTree[j, n]))
        
    q1 = calculate_risk_neutral_prob(u, d, R) 
    q2 = 1 - q1
    '''q1 and q2 are called risk neutral probabilities'''
    for i in range(n-1,-1,-1):
        for j in range(i+1):
            optionTree[j, i] = (q1 * optionTree[j, i+1] + q2 * optionTree[j+1, i+1])/R
   
                
    return optionTree

In [12]:
def american_put (T,n,sigma,r,c, S, K):
    '''parameters remain same as defined above'''
    u, d, R, q1, q2 = Parameter(T,n,sigma,r,c)
    optionTree = european_put(T,n,sigma,r,c, S, K)
    stockTree = bionomial_model(T,n,sigma,r,c, S, K)
    q1 = (R - d) / (u - d) 
    q2 = 1 - q1
    
    print(stockTree)
    print("risk neutral probability is " + str(q1))
    print(optionTree)
    
    #checking if early excercise is a good option 
    flag = 0 
    list = []
    for i in range(n-1,-1,-1):
        for j in range(i+1):
            optionTree[j, i] = max((q1 * optionTree[j, i+1] + q2 * optionTree[j+1, i+1])/R,   
                               (stockTree[j, i] - K))           
            #if the amount obtained by selling the option (Stock price - strike) is more than the oprion price, exercise early
            if (optionTree[j, i] - (stockTree[j, i] - K)) < 1e-10:
                flag += 1
                list.append(i)
    
    when = n
    if(flag):  when = list[-1]
    print("Can be exercised at time period " + str(when) + " with value " + str(optionTree[0][0]))
    return (optionTree[0,0], when)